In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import itertools
import os
import pandas as pd
#from google.colab import drive
import pickle
import random
#drive.mount('/content/drive')

np.random.seed(123)
torch.manual_seed(123)


## **Environment Class**

In [2]:
class BanditEnvironment:
    def __init__(self, difficulty, max_steps = 100):
        self.num_actions = 2  # Number of actions (arms/bandits)
        self.difficulty = difficulty
        self.max_steps = max_steps  # Maximum number of steps per episode
        self.reset()

    def reset(self):
        """Reset the environment to its initial state."""

        # Set brenoulli probabilities for difficulties
        if self.difficulty == 'easy':
            bandits_prob = np.random.choice([0.9, 0.1])
            self.bandits_prob = np.array([bandits_prob, 1 - bandits_prob])
        elif self.difficulty == 'medium':
            bandits_prob = np.random.choice([0.75, 0.25])
            self.bandits_prob = np.array([bandits_prob, 1 - bandits_prob])
        elif self.difficulty == 'hard':
            bandits_prob = np.random.choice([0.6, 0.4])
            self.bandits_prob = np.array([bandits_prob, 1 - bandits_prob])
        elif self.difficulty == 'uniform':
            bandits_prob = np.random.uniform()
            self.bandits_prob = np.array([bandits_prob, 1 - bandits_prob])
        else:  # Independent difficulty (random probabilities for each arm)
            self.bandits_prob = np.random.uniform(size=2)

        self.timestep = 0  # Initialize timestep
        self.last_action = None
        self.last_reward = 0
        return self.get_state()

    def step(self, action):
        """Take an action and return the next state, reward, and done flag."""
        self.timestep += 1
        self.last_action = action
        bandit_prob = self.bandits_prob[action]
        reward = 1 if np.random.uniform() < bandit_prob else 0
        self.last_reward = reward
        done = self.timestep >= self.max_steps  # End episode after 100 timesteps
        return self.get_state(), reward, done

    def get_state(self):
        """Return the current state: timestep, last action, and last reward."""
        one_hot_action = np.zeros(self.num_actions)
        if self.last_action is not None:
          one_hot_action[self.last_action] = 1
        return np.concatenate([np.array([self.timestep, self.last_reward], dtype=np.float32), one_hot_action])


In [3]:
def metrics(total_reward, bandits_prob, actions_chosen):

    # Cumulative Regret
    optimal_reward = np.max(bandits_prob) * 100
    cumulative_regret = optimal_reward - total_reward

    return cumulative_regret


In [4]:
class A2C_LSTM_Agent(nn.Module):
    def __init__(self, input_dim, hidden_size, num_actions):
        super(A2C_LSTM_Agent, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_size, batch_first=True)
        self.actor_layer = nn.Linear(hidden_size, num_actions)
        self.critic_layer = nn.Linear(hidden_size, 1)
        nn.init.xavier_uniform_(self.actor_layer.weight, gain=0.01)
        nn.init.xavier_uniform_(self.critic_layer.weight, gain=1.0)

    def forward(self, x, hidden_state):
        # x shape: (batch_size, seq_len, input_dim)
        lstm_out, hidden_state = self.lstm(x, hidden_state)
        lstm_out = lstm_out[:, -1, :]  # Use the output of the last timestep
        action_probs = F.softmax(self.actor_layer(lstm_out), dim=-1)
        value = self.critic_layer(lstm_out)
        return action_probs, value, hidden_state

    def init_hidden_state(self, batch_size=1):
        # Initialize the LSTM hidden and cell states
        return (torch.zeros(1, batch_size, self.hidden_size).to(device),
                torch.zeros(1, batch_size, self.hidden_size).to(device))

In [5]:
def train_agent(TRAIN = True, n_episodes = 25000, gamma = 0.99, learning_rate = 0.001, entropy_decay = False):

    stats = {'Episode Returns': [], 'Optimal Action': [], 'Bandits_Probs': [], 'Actions Taken': [], 'Cumulative Regret': [], 'Actions-Chosen List': []}

    for episode in tqdm(range(1, n_episodes + 1)):

        # Restart Constants, Environmentes and LSTM Hidden-State
        actions_chosen_list = [] if TRAIN == False else None
        actions_chosen = np.zeros(num_actions)
        state = env.reset()
        state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)  # Add batch and sequence dimensions
        hidden_state = agent.init_hidden_state()

        total_reward = 0
        done = False

        while not done:

            # Forward pass
            action_probs, value, hidden_state = agent(state, (hidden_state[0].detach(), hidden_state[1].detach()))

            # Sample action from probability distribution
            action_dist = torch.distributions.Categorical(action_probs)
            action = action_dist.sample()
            log_prob = action_dist.log_prob(action)

            # Take action in the environment
            next_state, reward, done = env.step(action.item())
            total_reward += reward

            # Prepare next state
            next_state = torch.FloatTensor(next_state).unsqueeze(0).unsqueeze(0).to(device)

            # Trainning
            if TRAIN == True:

                # Compute TD target and advantage
                with torch.no_grad():
                    _, next_value, _ = agent(next_state, hidden_state) #if not done else (None, torch.tensor(0.0), None)
                    td_target = reward + gamma * next_value
                advantage = td_target - value

                # Compute losses
                actor_loss = -log_prob * advantage.detach()
                critic_loss = F.mse_loss(value, td_target)
                entropy_loss = -action_dist.entropy().mean()
                entropy_coef = max(0.005, 0.1 * (1 - episode / n_episodes)) if entropy_decay == True else 0.005

                loss = actor_loss + 0.05 * critic_loss + entropy_coef * entropy_loss

                # Optimize the network
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            else:
                actions_chosen_list.append(action.item())

            # Move to the next state
            state = next_state

            # Add action
            actions_chosen[action.item()] += 1

        # Cumulative Regret:
        cumulative_regret_value = metrics(total_reward, env.bandits_prob, actions_chosen)

        #Save Episode stats
        stats['Episode Returns'].append(total_reward)
        stats['Optimal Action'].append(np.argmax(env.bandits_prob))
        stats['Bandits_Probs'].append(env.bandits_prob)
        stats['Actions Taken'].append(actions_chosen)
        stats['Actions-Chosen List'].append(actions_chosen_list)
        stats['Cumulative Regret'].append(metrics(total_reward, env.bandits_prob, actions_chosen))

        if len(stats['Cumulative Regret']) >= 100:
            mean_last_100_regret = np.mean(stats['Cumulative Regret'][-100:])
        else:
            mean_last_100_regret = np.mean(stats['Cumulative Regret'])


        if episode % 1000 == 0:
            print(f"Ep {episode}/{n_episodes}, Opt. Action: {np.argmax(env.bandits_prob)}, Reward: {total_reward}, Cumulative-Regret: {cumulative_regret_value}, AVG100-Regret: {mean_last_100_regret}")

    return stats



In [6]:
def save_train_stats(stats, model_name, train_difficulty, discount_factor_gamma, learning_rate, entropy_decay):

    stats = pd.DataFrame(stats)

    stats['model_name'] = model_name
    stats['train_difficulty'] = train_difficulty
    stats['gamma'] = discount_factor_gamma
    stats['lr'] = learning_rate
    stats['entropy_decay'] = entropy_decay
    stats.to_pickle(google_drive_folder + '/' + 'train_stats' + '/' + model_name + '.pkl')

def save_test_stats(stats, model_name, train_difficulty, test_difficulty, discount_factor_gamma, learning_rate, entropy_decay):

    stats = pd.DataFrame(stats)

    stats['model_name'] = model_name
    stats['train_difficulty'] = train_difficulty
    stats['gamma'] = discount_factor_gamma
    stats['lr'] = learning_rate
    stats['test_difficulty'] = test_difficulty
    stats['test_difficulty'] = test_difficulty
    stats['entropy_decay'] = entropy_decay


    model_name = model_name + f'__TestDiff_{test_difficulty}'
    stats.to_pickle(google_drive_folder + '/' + 'test_stats' + '/' + model_name + '.pkl')



In [7]:
# Set Google Drive Folder
google_drive_folder = './Monografia/Exp1-2/'
os.makedirs(google_drive_folder + 'models', exist_ok=True)
os.makedirs(google_drive_folder + 'train_stats', exist_ok=True)
os.makedirs(google_drive_folder + 'test_stats', exist_ok=True)

# Hyperparameters
GAMMA_LIST = [0.7, 0.9, 0.99]
LR_LIST = [0.001, 0.0001]
ENTROPY_DECAY = [False, True]
TRAIN_DIFFICULTY = ['easy']
TEST_DIFFICULTY = ['easy', 'medium', 'hard', 'uniform', 'independent']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hidden_size = 48
max_steps = 100
n_episodes_train = 25000
n_episodes_test= 300

hyperparameters_done = [
    (0.7, 0.0001, True),
    (0.9, 0.0001, True),
    (0.9, 0.0001, False),
    (0.9, 0.001, True),
    (0.7, 0.001, False),
    (0.9, 0.001, False)
]


# Process
for train_difficulty in TRAIN_DIFFICULTY:

  hyperparameters = list(itertools.product(GAMMA_LIST, LR_LIST, ENTROPY_DECAY)) # 100 in paper
  random.shuffle(hyperparameters)

  for remove in hyperparameters_done:
     hyperparameters.remove(remove)

  for i, (gamma, learning_rate, entropy_decay) in enumerate(hyperparameters):

      print(f'\nTRAIN DIFFICULTY {train_difficulty}\nHyperparameter {i+1}: GAMMA {gamma} - LR {learning_rate} - Entropy Decay {entropy_decay}')

      # Environment and agent setup
      env = BanditEnvironment(difficulty = train_difficulty, max_steps = max_steps)
      num_actions = env.num_actions
      input_dim = 1 + num_actions + 1  # Timestep, past action oh, past reward
      agent = A2C_LSTM_Agent(input_dim, hidden_size, num_actions).to(device)
      optimizer = optim.Adam(agent.parameters(), lr=learning_rate)

      # Train
      stats = train_agent(TRAIN = True, n_episodes = n_episodes_train, gamma = gamma, learning_rate = learning_rate, entropy_decay = entropy_decay)

      # Save Model and Train Stats
      agent.eval()
      model_name = f'TrainDiff_{train_difficulty}__LR_{str(learning_rate).replace(".","_")}__GAMMA_{str(gamma).replace(".","_")}__EntropyDecay_{str(entropy_decay)}'
      torch.save(agent.state_dict(), google_drive_folder + 'models/'+ model_name + '.pth')
      save_train_stats(stats, model_name, train_difficulty, gamma, learning_rate, entropy_decay)

      for test_difficulty in TEST_DIFFICULTY:

          print(f'\nTEST {test_difficulty}:')

          # Test and save stats
          stats = train_agent(TRAIN = False, n_episodes = n_episodes_test, gamma = gamma, learning_rate = learning_rate, entropy_decay = entropy_decay)
          save_test_stats(stats, model_name, train_difficulty, test_difficulty, gamma, learning_rate, entropy_decay)




TRAIN DIFFICULTY easy
Hyperparameter 1: GAMMA 0.7 - LR 0.0001 - Entropy Decay False


  4%|▍         | 1000/25000 [14:29<6:17:01,  1.06it/s]

Ep 1000/25000, Opt. Action: 1, Reward: 83, Cumulative-Regret: 7.0, AVG100-Regret: 5.26


  8%|▊         | 2000/25000 [28:24<5:27:59,  1.17it/s]

Ep 2000/25000, Opt. Action: 1, Reward: 91, Cumulative-Regret: -1.0, AVG100-Regret: 3.89


...

100%|██████████| 300/300 [01:01<00:00,  4.87it/s]



TEST uniform:


100%|██████████| 300/300 [01:01<00:00,  4.86it/s]



TEST independent:


100%|██████████| 300/300 [01:01<00:00,  4.87it/s]
